In [1]:
import re, time
import pandas as pd
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack import Pipeline, Document

import os
from getpass import getpass


/home/sanka/anaconda3/envs/tf-env-ultimate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sanka/anaconda3/envs/tf-env-ultimate/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-07-02 15:52:27.786925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 15:52:28.966323: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/sanka/anaconda3/envs/tf-env-ultimate/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 

In [56]:
def process_log_file(file_path)-> pd:
    pattern = r'(?P<timestamp>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}) \((?P<agent>[\w.-]+[^)]+)\) (?P<debug>[A-Za-z_\.]+) (?P<level>[A-Z]+): (?P<message>.*)'
    #pattern = r'^(?P<timestamp>[\d-]+\s[\d:,]+)\s\((?P<module>[\w.-]+)\s(?P<process_id>\d+)\s\[(?P<thread_id>\d+)\]\)\s(?P<source>[\w.]+)\s(?P<log_level>\w+):\s(?P<message>.+)$'
    
    data = {
        "timestamp": [],
        "agent": [],
        "debug": [],
        "level": [],
        "message": []
    }

    with open(file_path, 'r') as file:
        current_log = []
        line_count = 0

        for line in file:
            # Check if the line matches the start of a new log entry
            if re.match(pattern, line):
                if current_log:
                    full_log_entry = "\n".join(current_log)
                    match = re.match(pattern, full_log_entry, re.DOTALL)
                    if match:
                        data["timestamp"].append(match.group('timestamp'))
                        data["agent"].append(match.group('agent'))
                        data["debug"].append(match.group('debug'))
                        data["level"].append(match.group('level'))
                        data["message"].append(match.group('message').strip())

                current_log = [line.strip()]
                line_count = 1
            else:
                if line_count < 4:
                    current_log.append(line.strip())
                    line_count += 1

        # Append the last log entry
        if current_log:
            full_log_entry = "\n".join(current_log)
            match = re.match(pattern, full_log_entry, re.DOTALL)
            if match:
                data["timestamp"].append(match.group('timestamp'))
                data["agent"].append(match.group('agent'))
                data["debug"].append(match.group('debug'))
                data["level"].append(match.group('level'))
                data["message"].append(match.group('message').strip())

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    return df

In [57]:
from haystack.utils import Secret
log2=process_log_file('NIRE_540_ASSETS.log')
print(log2.head)


<bound method NDFrame.head of Empty DataFrame
Columns: [timestamp, agent, debug, level, message]
Index: []>


In [ ]:
no_of_rows=30
# Create a list to store the context and prompts
data2 = []
client2 = OpenAIGenerator(model="gpt-3.5-turbo", generation_kwargs={"max_tokens": 4000})
# Convert each row to a JSON object suitable for ChatGPT prompts
current_context2=f"\n"
prompt2=''
row_count=0
for _, row in log2.iterrows():
    context2 = f"Timestamp: {row['timestamp']}\nAgent: {row['agent']}\nDebug: {row['debug']}\nLevel: {row['level']}\nMessage: {row['message']}"
    if row_count<=no_of_rows:
        current_context2=current_context2+f"\n"+context2
        
    else:
        prompt2 = f"provide a detaled technical summery of the system for the system log in this given context given bellow . include the time window, how many agents had events. how many error event during the time period and discribe errors:\n{current_context2}"
        #print(prompt,"UUUUUUUUUUUUUUUUUUUUUUUUUUUUUU")
        response=client2.run(prompt2)
        print(len(data2),len(prompt2),response['replies'])
        gptresponse2=response['replies']
        data2.append({"response": gptresponse2, "prompt": prompt2,"context":current_context2})
        current_context2=f"\n"
        time.sleep(10)
        row_count=0
    row_count=row_count+1


In [10]:
 df2 = pd.DataFrame(data2)
 df2.to_parquet('540log_mass.parquet', index=False)

In [12]:
df = pd.read_parquet('540log_mass.parquet')

# Display the DataFrame
print(df)

                                              response  \
0    [The time window for the system log is from 16...   
1    [The system log contains events recorded withi...   
2    [The system log contains events related to the...   
3    [The system log contains multiple entries from...   
4    [Time Window: 2024-06-18 16:56:56 to 2024-06-1...   
..                                                 ...   
661  [The system log contains multiple timestamps w...   
662  [Time window: 2024-06-18 17:15:06 to 2024-06-1...   
663  [The system log contains events from the platf...   
664  [The system log provided captures events from ...   
665  [The system log contains multiple events from ...   

                                                prompt  \
0    provide a detaled technical summery of the sys...   
1    provide a detaled technical summery of the sys...   
2    provide a detaled technical summery of the sys...   
3    provide a detaled technical summery of the sys...   
4    provide 